# Supply Chain Management Simualtion

In [380]:
# import desired packages 
from pulp import *
import numpy as np # mainly for matrix manipulation
import pandas as pd # mainly for dataframe manipulation
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from pandas import to_datetime
from statsmodels.tsa.api import ExponentialSmoothing
from sklearn.linear_model import LinearRegression
from numpy import ones,vstack
from numpy.linalg import lstsq
from datetime import datetime
import math
import cvxpy as cp
import warnings
warnings.filterwarnings("ignore")

## Long-term strategic decisions

### Mixed-Integer Linear Programming Model

In [497]:
# Decisions Variables

# Binary Variables 
# we will start by defining two sets of binary variables that would tell which warehouse and factory to build
# since calopeia starts the game with a factory and a warehouse it will not be included in these sets
country_list = ['calopeia', 'sorange', 'entworpe', 'tyran', 'fardo']
warehouse_vars = LpVariable.dicts("warehouse", country_list[1:], lowBound=0, upBound=1, cat='Integer')
factory_vars = LpVariable.dicts("factory", country_list[1:], lowBound=0, upBound=1, cat='Integer')

# next we will define the capacity amount to add in each factories (if there is gonna be one)
capacity_vars = LpVariable.dicts("capacity", country_list, lowBound=0, cat='Integer')

# Continuous Variables
# now we can set continuous variables that will determine the numbers of units transported for max profit
# first the units transported from one factory to one warehouse by mail and truck
# c represents calopeia, s: sorange, e: entworpe, t: tyran and f: fardo
trajectories = ['calopeia_calopeia', 'calopeia_sorange', 'calopeia_entworpe', 'calopeia_tyran', 'calopeia_fardo', \
                'sorange_calopeia', 'sorange_sorange', 'sorange_entworpe', 'sorange_tyran', 'sorange_fardo', \
                'entworpe_calopeia', 'entworpe_sorange', 'entworpe_entworpe', 'entworpe_tyran', 'entworpe_fardo', \
                'tyran_calopeia', 'tyran_sorange', 'tyran_entworpe', 'tyran_tyran', 'tyran_fardo', \
                'fardo_calopeia', 'fardo_sorange', 'fardo_entworpe', 'fardo_tyran', 'fardo_fardo']

mail1_vars = LpVariable.dicts("mail1", trajectories, lowBound=0, cat='Integer')
truck_vars = LpVariable.dicts("truck", trajectories, lowBound=0, cat='Integer')

truck_weights = [75, 100, 100, 100, 225,
                100, 75, 100, 100, 225,
                100, 100, 75, 100, 225,
                100, 100, 100, 75, 225,
                225, 225, 225, 225, 75]

mail_weights =  [150, 200, 200, 200, 400,
                200, 150, 200, 200, 400,
                200, 200, 150, 200, 400,
                200, 200, 200, 150, 400,
                400, 400, 400, 400, 150]

# second, we will define the units transported from the warehouses to the markets by mail
mail2_vars = LpVariable.dicts("mail2", trajectories, lowBound=0, cat='Integer')

demand = [avg_demand_calopeia, avg_demand_sorange, avg_demand_entworpe2[0], avg_demand_tyran, avg_demand_fardo]

# Objective Function

# initialise the objective function
max_profit = LpProblem("Supply_Chain_Game", LpMaximize)

# calculate the potential revenues and costs
# revenue set as the price of the one unit times the number of drums that can be produced every day
revenue = 1450 * (51100 + 640 * lpSum([capacity_vars[i] for i in capacity_vars]))

# cost set as the addition of cost of production units, cost of factories and warehouses, cost of additional capacity
# cost of transport to warehouse, and cost of transport to customers.  
cost =   (500000*lpSum([factory_vars[i] for i in factory_vars]) \
       + 100000*lpSum([factory_vars[i] for i in factory_vars]) \
       + 50000*lpSum([capacity_vars[i] for i in factory_vars]) \
       + lpSum([truck_vars[i]*j for i,j in zip(truck_vars, truck_weights)]) \
       + lpSum([mail1_vars[i]*j for i,j in zip(mail1_vars, mail_weights)])  \
       + lpSum([mail2_vars[i]*j for i,j in zip(mail2_vars, mail_weights)]))
        
# compile the objective function
max_profit += (revenue - cost)


# Constraints

# set the constraints to our problem
# the first set ensure that all the units produced are sent to the warehouses from each factories
# from calopeia
max_profit += lpSum([i+j for i,j in zip([i for i in mail1_vars.values()][:5], \
                                        [i for i in truck_vars.values()][:5])]) \
            == 51100 + 640*[i for i in capacity_vars.values()][0]

# from sorange
max_profit += lpSum([i+j for i,j in zip([i for i in mail1_vars.values()][5:10], \
                                        [i for i in truck_vars.values()][5:10])]) \
            == 640*[i for i in capacity_vars.values()][1]

# from entworpe
max_profit += lpSum([i+j for i,j in zip([i for i in mail1_vars.values()][10:15], \
                                        [i for i in truck_vars.values()][10:15])]) \
            == 640*[i for i in capacity_vars.values()][2]

# from tyran
max_profit += lpSum([i+j for i,j in zip([i for i in mail1_vars.values()][15:20], \
                                        [i for i in truck_vars.values()][15:20])]) \
            == 640*[i for i in capacity_vars.values()][3]

# from fardo
max_profit += lpSum([i+j for i,j in zip([i for i in mail1_vars.values()][20:25], \
                                        [i for i in truck_vars.values()][20:25])]) \
            == 640*[i for i in capacity_vars.values()][4]

# the second set ensure that all the units shipped to the warehouses are distributed to the markets
# from calopeia
max_profit += lpSum([i+j for i,j in zip([i for i in mail1_vars.values()][0::5], \
                                        [i for i in truck_vars.values()][0::5])]) \
            == lpSum([i for i in [i for i in mail2_vars.values()][:5]])

# from sorange
max_profit += lpSum([i+j for i,j in zip([i for i in mail1_vars.values()][1::5], \
                                        [i for i in truck_vars.values()][1::5])]) \
            == lpSum([i for i in [i for i in mail2_vars.values()][5:10]])

# from entworpe
max_profit += lpSum([i+j for i,j in zip([i for i in mail1_vars.values()][2::5], \
                                        [i for i in truck_vars.values()][2::5])]) \
            == lpSum([i for i in [i for i in mail2_vars.values()][10:15]])

# from tyran
max_profit += lpSum([i+j for i,j in zip([i for i in mail1_vars.values()][3::5], \
                                        [i for i in truck_vars.values()][3::5])]) \
            == lpSum([i for i in [i for i in mail2_vars.values()][15:20]])

# from fardo
max_profit += lpSum([i+j for i,j in zip([i for i in mail1_vars.values()][4::5], \
                                        [i for i in truck_vars.values()][4::5])]) \
            == lpSum([i for i in [i for i in mail2_vars.values()][20:25]])

# the third set of constraints aim to limit the produced orders to match the customers demand
# from calopeia
max_profit += lpSum([i for i in [i for i in mail2_vars.values()][0::5]]) \
            <= demand[0]

# from sorange
max_profit += lpSum([i for i in [i for i in mail2_vars.values()][1::5]]) \
            <= demand[1]

# from entworpe
max_profit += lpSum([i for i in [i for i in mail2_vars.values()][2::5]]) \
            <= demand[2]

# from tyran
max_profit += lpSum([i for i in [i for i in mail2_vars.values()][3::5]]) \
            <= demand[3]

# from fardo
max_profit += lpSum([i for i in [i for i in mail2_vars.values()][4::5]]) \
            <= demand[4]

# the fourth set of constraints link the shipments from the warehouses depending on if the warehouse if built
# from sorange
max_profit += lpSum([i for i in [i for i in mail2_vars.values()][5:10]]) \
            <= 1000000 * [i for i in warehouse_vars.values()][0]

# from entworpe
max_profit += lpSum([i for i in [i for i in mail2_vars.values()][10:15]]) \
            <= 1000000 * [i for i in warehouse_vars.values()][1]

# from tyran
max_profit += lpSum([i for i in [i for i in mail2_vars.values()][15:20]]) \
            <= 1000000 * [i for i in warehouse_vars.values()][2]

# from fardo
max_profit += lpSum([i for i in [i for i in mail2_vars.values()][20:25]]) \
            <= 1000000 * [i for i in warehouse_vars.values()][3]

# the final set of constraint link the potential capacity of an unbuilt factory to its construction
for j in range(0,4):
    max_profit += [i for i in capacity_vars.values()][j+1] <= [i for i in factory_vars.values()][j]*1000

solver = CPLEX_CMD(path='/Applications/CPLEX_Studio221/cplex/bin/x86-64_osx/cplex')
max_profit.solve(solver)


Welcome to IBM(R) ILOG(R) CPLEX(R) Interactive Optimizer 22.1.0.0
  with Simplex, Mixed Integer & Barrier Optimizers
5725-A06 5725-A29 5724-Y48 5724-Y49 5724-Y54 5724-Y55 5655-Y21
Copyright IBM Corp. 1988, 2022.  All Rights Reserved.

Type 'help' for a list of available commands.
Type 'help' followed by a command name for more
information on commands.

CPLEX> Problem '/var/folders/24/642ljpss6bd8ljpp58f9qjy40000gn/T/6d74fb2da55c4ce2b080a2ef737766b1-pulp.lp' read.
Read time = 0.00 sec. (0.01 ticks)
CPLEX> Version identifier: 22.1.0.0 | 2022-03-09 | 1a383f8ce
Tried aggregator 1 time.
MIP Presolve eliminated 4 rows and 29 columns.
MIP Presolve modified 4 coefficients.
Reduced MIP has 19 rows, 59 columns, and 113 nonzeros.
Reduced MIP has 4 binaries, 55 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.11 ticks)
Found incumbent of value -1.4913550e+07 after 0.00 sec. (0.31 ticks)
Probing time = 0.00 sec. (0.00 ticks)
Tried aggregator 1 time.
Detecting symmetries...
Reduced 

1

In [498]:
print("Status:", LpStatus[max_profit.status])
for v in max_profit.variables():
    if v.varValue > 0:
        print(v.name, "=", v.varValue)

Status: Optimal
capacity_calopeia = 75.0
capacity_fardo = 17.0
factory_fardo = 1.0
mail2_calopeia_calopeia = 27879.0
mail2_entworpe_entworpe = 8250.0
mail2_fardo_fardo = 10984.0
mail2_sorange_sorange = 48999.0
mail2_tyran_tyran = 13868.0
truck_calopeia_calopeia = 27879.0
truck_calopeia_entworpe = 8250.0
truck_calopeia_fardo = 104.0
truck_calopeia_sorange = 48999.0
truck_calopeia_tyran = 13868.0
truck_fardo_fardo = 10880.0
warehouse_entworpe = 1.0
warehouse_fardo = 1.0
warehouse_sorange = 1.0
warehouse_tyran = 1.0


## Short-term strategic decisions

### Monthly POQ for all regions

#### Non-linear programming for monthly POQ

Since the data is very seasonal for Calopeia, it would be best to determine the periodic order quantity for each month as the data varies a lot from month to month. 

In [397]:
calopeia_demand2 = calopeia_demand.loc[:, :]
calopeia_demand2['date'] = pd.to_datetime(calopeia_demand2.index, unit='D')
calopeia_demand2 = calopeia_demand2.set_index('date')
calopeia_demand2 = calopeia_demand2.groupby(by=[calopeia_demand2.index.year, calopeia_demand2.index.month]).sum()
calopeia_demand2 = calopeia_demand2.iloc[:-1, :]

In [398]:
demand = calopeia_demand2.Calopeia.values
months = [i for i in range(1,25)]
holding_cost = 100/12

quantity = cp.Variable(24)
order = cp.Variable(24, boolean=True)
start_inventory = cp.Variable(24)
end_inventory = cp.Variable(24)

cost = 0
for i in range(24):
    cost += (quantity[i]*1000 + 1500)*order[i] + holding_cost*end_inventory[i]

constraints = []    
for i in range(23):
    constraints.append(end_inventory[i] == start_inventory[i+1])

for i in range(24):
    constraints.append(end_inventory[i] == start_inventory[i] + quantity[i] - demand[i])
    constraints.append(10000*order[i] >= quantity[i])
    
obj = cp.Minimize(sum(cost))
prob = cp.Problem(obj, constraints)
prob.solve()
for variable in prob.variables():
    print("Variable %s: value %s" % (variable.name(), variable.value))

Variable var28332: value [   0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0. -325.]
Variable var28331: value [313.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
Variable var28329: value [   0.  454. 1007. 1356. 1915. 2123. 2215. 1938. 1545.  888.  431.  315.
  353.  500.  977. 1194. 1676. 2033. 2091. 1982. 1491.  917.  562.    0.]
Variable var28330: value [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


#### MILP programming for monthly POQ - version 1

#### MILP for POQ - Final version

In [499]:
months = [i for i in range(1,25)]
holding_price = 100/12

# monthly demand for each region
demand_calopeia = calopeia_demand2.Calopeia.values

sorange_demand2 = pd.DataFrame(np.concatenate((y_pred, y_pred2)), index =[i for i in range(1,731)], \
                              columns =['sum'])
sorange_demand2['date'] = pd.to_datetime(sorange_demand2.index, unit='D')
sorange_demand2 = sorange_demand2.set_index('date')
sorange_demand2 = sorange_demand2.groupby(by=[sorange_demand2.index.year, sorange_demand2.index.month]).sum()
sorange_demand2 = sorange_demand2.iloc[:-1, :]
demand_sorange = sorange_demand2['sum'].values

demand_tyran0 = [avg_tyran[0]]*700
for i in y_pred_tyran:
    demand_tyran0.append(i)
demand_tyran = demand_tyran0[:]

demand_fardo0 = [avg_fardo[0]]*700
for i in y_pred_fardo:
    demand_fardo0.append(i)
demand_fardo = demand_fardo0[:]


# number of drums produced in Caleopia and Fardo factories
caleopia_drums_quantity_vars = LpVariable.dicts("caleopia_drums_quantity", months, lowBound=0, cat='Integer')
fardo_drums_quantity_vars = LpVariable.dicts("fardo_drums_quantity", months, lowBound=0, cat='Integer')


# number of trucks sent from caleopia factory
truck_quantity_from_caleopia_to_caleopia_vars = LpVariable.dicts("truck_quantity_from_caleopia_to_caleopia", months, \
                                                                 lowBound=0, cat='Integer')
truck_quantity_from_caleopia_to_fardo_vars = LpVariable.dicts("truck_quantity_from_caleopia_to_fardo", months, \
                                                                 lowBound=0, cat='Integer')
truck_quantity_from_caleopia_to_tyran_vars = LpVariable.dicts("truck_quantity_from_caleopia_to_tyran", months, \
                                                                 lowBound=0, cat='Integer')
truck_quantity_from_caleopia_to_entworpe_vars = LpVariable.dicts("truck_quantity_from_caleopia_to_entworpe", months, \
                                                                 lowBound=0, cat='Integer')
truck_quantity_from_caleopia_to_sorange_vars = LpVariable.dicts("truck_quantity_from_caleopia_to_sorange", months, \
                                                                lowBound=0, cat='Integer')

# number of trucks sent from fardo factory
truck_quantity_from_fardo_to_caleopia_vars = LpVariable.dicts("truck_quantity_from_fardo_to_caleopia", months, \
                                                                 lowBound=0, cat='Integer')
truck_quantity_from_fardo_to_fardo_vars = LpVariable.dicts("truck_quantity_from_fardo_to_fardo", months, \
                                                                 lowBound=0, cat='Integer')
truck_quantity_from_fardo_to_tyran_vars = LpVariable.dicts("truck_quantity_from_fardo_to_tyran", months, \
                                                                 lowBound=0, cat='Integer')
truck_quantity_from_fardo_to_entworpe_vars = LpVariable.dicts("truck_quantity_from_fardo_to_entworpe", months, \
                                                                 lowBound=0, cat='Integer')
truck_quantity_from_fardo_to_sorange_vars = LpVariable.dicts("truck_quantity_from_fardo_to_sorange", months, \
                                                             lowBound=0, cat='Integer')

# number of drums sent by mail from caleopia factory
quantity_from_caleopia_to_caleopia_by_mail_vars = LpVariable.dicts("quantity_from_caleopia_to_caleopia_by_mail", \
                                                                   months, lowBound=0, cat='Integer')
quantity_from_caleopia_to_fardo_by_mail_vars = LpVariable.dicts("quantity_from_caleopia_to_fardo_by_mail", \
                                                                   months, lowBound=0, cat='Integer')
quantity_from_caleopia_to_tyran_by_mail_vars = LpVariable.dicts("quantity_from_caleopia_to_tyran_by_mail", \
                                                                   months, lowBound=0, cat='Integer')
quantity_from_caleopia_to_entworpe_by_mail_vars = LpVariable.dicts("quantity_from_caleopia_to_entworpe_by_mail", \
                                                                   months, lowBound=0, cat='Integer')
quantity_from_caleopia_to_sorange_by_mail_vars = LpVariable.dicts("quantity_from_caleopia_to_sorange_by_mail", \
                                                                   months, lowBound=0, cat='Integer')

# number of drums sent by mail from fardo factory
quantity_from_fardo_to_caleopia_by_mail_vars = LpVariable.dicts("quantity_from_fardo_to_caleopia_by_mail", \
                                                                   months, lowBound=0, cat='Integer')
quantity_from_fardo_to_fardo_by_mail_vars = LpVariable.dicts("quantity_from_fardo_to_fardo_by_mail", \
                                                                   months, lowBound=0, cat='Integer')
quantity_from_fardo_to_tyran_by_mail_vars = LpVariable.dicts("quantity_from_fardo_to_tyran_by_mail", \
                                                                   months, lowBound=0, cat='Integer')
quantity_from_fardo_to_entworpe_by_mail_vars = LpVariable.dicts("quantity_from_fardo_to_entworpe_by_mail", \
                                                                   months, lowBound=0, cat='Integer')
quantity_from_fardo_to_sorange_by_mail_vars = LpVariable.dicts("quantity_from_fardo_to_sorange_by_mail", \
                                                                   months, lowBound=0, cat='Integer')

# number of drums sent by truck from caleopia factory
quantity_from_caleopia_to_caleopia_by_truck_vars = LpVariable.dicts("quantity_from_caleopia_to_caleopia_by_truck", \
                                                                   months, lowBound=0, cat='Integer')
quantity_from_caleopia_to_fardo_by_truck_vars = LpVariable.dicts("quantity_from_caleopia_to_fardo_by_truck", \
                                                                   months, lowBound=0, cat='Integer')
quantity_from_caleopia_to_tyran_by_truck_vars = LpVariable.dicts("quantity_from_caleopia_to_tyran_by_truck", \
                                                                   months, lowBound=0, cat='Integer')
quantity_from_caleopia_to_entworpe_by_truck_vars = LpVariable.dicts("quantity_from_caleopia_to_entworpe_by_truck", \
                                                                   months, lowBound=0, cat='Integer')
quantity_from_caleopia_to_sorange_by_truck_vars = LpVariable.dicts("quantity_from_caleopia_to_sorange_by_truck", \
                                                                   months, lowBound=0, cat='Integer')

# number of drums sent by truck from fardo factory
quantity_from_fardo_to_caleopia_by_truck_vars = LpVariable.dicts("quantity_from_fardo_to_caleopia_by_truck", \
                                                                   months, lowBound=0, cat='Integer')
quantity_from_fardo_to_fardo_by_truck_vars = LpVariable.dicts("quantity_from_fardo_to_fardo_by_truck", \
                                                                   months, lowBound=0, cat='Integer')
quantity_from_fardo_to_tyran_by_truck_vars = LpVariable.dicts("quantity_from_fardo_to_tyran_by_truck", \
                                                                   months, lowBound=0, cat='Integer')
quantity_from_fardo_to_entworpe_by_truck_vars = LpVariable.dicts("quantity_from_fardo_to_entworpe_by_truck", \
                                                                   months, lowBound=0, cat='Integer')
quantity_from_fardo_to_sorange_by_truck_vars = LpVariable.dicts("quantity_from_fardo_to_sorange_by_truck", \
                                                                   months, lowBound=0, cat='Integer')

# binary variable if ordered by mail from caleopia 
from_caleopia_to_caleopia_by_mail_vars = LpVariable.dicts("from_caleopia_to_caleopia_by_mail?", \
                                                                   months, lowBound=0, upBound = 1, cat='Integer')
from_caleopia_to_fardo_by_mail_vars = LpVariable.dicts("from_caleopia_to_fardo_by_mail?", \
                                                                   months, lowBound=0, upBound = 1, cat='Integer')
from_caleopia_to_tyran_by_mail_vars = LpVariable.dicts("from_caleopia_to_tyran_by_mail?", \
                                                                   months, lowBound=0, upBound = 1, cat='Integer')
from_caleopia_to_entworpe_by_mail_vars = LpVariable.dicts("from_caleopia_to_entworpe_by_mail?", \
                                                                   months, lowBound=0, upBound = 1, cat='Integer')
from_caleopia_to_sorange_by_mail_vars = LpVariable.dicts("from_caleopia_to_sorange_by_mail?", \
                                                                   months, lowBound=0, upBound = 1, cat='Integer')

# binary variable if ordered by mail from fardo 
from_fardo_to_caleopia_by_mail_vars = LpVariable.dicts("from_fardo_to_caleopia_by_mail?", \
                                                                   months, lowBound=0, upBound = 1, cat='Integer')
from_fardo_to_fardo_by_mail_vars = LpVariable.dicts("from_fardo_to_fardo_by_mail?", \
                                                                   months, lowBound=0, upBound = 1, cat='Integer')
from_fardo_to_tyran_by_mail_vars = LpVariable.dicts("from_fardo_to_tyran_by_mail?", \
                                                                   months, lowBound=0, upBound = 1, cat='Integer')
from_fardo_to_entworpe_by_mail_vars = LpVariable.dicts("from_fardo_to_entworpe_by_mail?", \
                                                                   months, lowBound=0, upBound = 1, cat='Integer')
from_fardo_to_sorange_by_mail_vars = LpVariable.dicts("from_fardo_to_sorange_by_mail?", \
                                                                   months, lowBound=0, upBound = 1, cat='Integer')

# caleopia start and end inventory
caleopia_start_inventory_vars = LpVariable.dicts("caleopia_start_inventory", months, lowBound=0, cat='Integer')
caleopia_end_inventory_vars = LpVariable.dicts("caleopia_end_inventory", months, lowBound=0, cat='Integer')

# fardo start and end inventory
fardo_start_inventory_vars = LpVariable.dicts("fardo_start_inventory", months, lowBound=0, cat='Integer')
fardo_end_inventory_vars = LpVariable.dicts("fardo_end_inventory", months, lowBound=0, cat='Integer')

# tyran start and end inventory
tyran_start_inventory_vars = LpVariable.dicts("tyran_start_inventory", months, lowBound=0, cat='Integer')
tyran_end_inventory_vars = LpVariable.dicts("tyran_end_inventory", months, lowBound=0, cat='Integer')

# entworpe start and end inventory
entworpe_start_inventory_vars = LpVariable.dicts("entworpe_start_inventory", months, lowBound=0, cat='Integer')
entworpe_end_inventory_vars = LpVariable.dicts("entworpe_end_inventory", months, lowBound=0, cat='Integer')

# sorange start and end inventory
sorange_start_inventory_vars = LpVariable.dicts("sorange_start_inventory", months, lowBound=0, cat='Integer')
sorange_end_inventory_vars = LpVariable.dicts("sorange_end_inventory", months, lowBound=0, cat='Integer')

# quantities sent from caleopia warehouse
quantity_from_caleopia_to_caleopia_final_vars = LpVariable.dicts("quantity_from_caleopia_to_caleopia_final", \
                                                                   months, lowBound=0, cat='Integer')
quantity_from_caleopia_to_fardo_final_vars = LpVariable.dicts("quantity_from_caleopia_to_fardo_final", \
                                                                   months, lowBound=0, cat='Integer')
quantity_from_caleopia_to_tyran_final_vars = LpVariable.dicts("quantity_from_caleopia_to_tyran_final", \
                                                                   months, lowBound=0, cat='Integer')
quantity_from_caleopia_to_entworpe_final_vars = LpVariable.dicts("quantity_from_caleopia_to_entworpe_final", \
                                                                   months, lowBound=0, cat='Integer')
quantity_from_caleopia_to_sorange_final_vars = LpVariable.dicts("quantity_from_caleopia_to_sorange_final", \
                                                                   months, lowBound=0, cat='Integer')

# quantities sent from fardo warehouse
quantity_from_fardo_to_caleopia_final_vars = LpVariable.dicts("quantity_from_fardo_to_caleopia_final", \
                                                                   months, lowBound=0, cat='Integer')
quantity_from_fardo_to_fardo_final_vars = LpVariable.dicts("quantity_from_fardo_to_fardo_final", \
                                                                   months, lowBound=0, cat='Integer')
quantity_from_fardo_to_tyran_final_vars = LpVariable.dicts("quantity_from_fardo_to_tyran_final", \
                                                                   months, lowBound=0, cat='Integer')
quantity_from_fardo_to_entworpe_final_vars = LpVariable.dicts("quantity_from_fardo_to_entworpe_final", \
                                                                   months, lowBound=0, cat='Integer')
quantity_from_fardo_to_sorange_final_vars = LpVariable.dicts("quantity_from_fardo_to_sorange_final", \
                                                                   months, lowBound=0, cat='Integer')

# quantities sent from sorange warehouse
quantity_from_sorange_to_caleopia_final_vars = LpVariable.dicts("quantity_from_sorange_to_caleopia_final", \
                                                                   months, lowBound=0, cat='Integer')
quantity_from_sorange_to_fardo_final_vars = LpVariable.dicts("quantity_from_sorange_to_fardo_final", \
                                                                   months, lowBound=0, cat='Integer')
quantity_from_sorange_to_tyran_final_vars = LpVariable.dicts("quantity_from_sorange_to_tyran_final", \
                                                                   months, lowBound=0, cat='Integer')
quantity_from_sorange_to_entworpe_final_vars = LpVariable.dicts("quantity_from_sorange_to_entworpe_final", \
                                                                   months, lowBound=0, cat='Integer')
quantity_from_sorange_to_sorange_final_vars = LpVariable.dicts("quantity_from_sorange_to_sorange_final", \
                                                                   months, lowBound=0, cat='Integer')

# quantities sent from entworpe warehouse
quantity_from_entworpe_to_caleopia_final_vars = LpVariable.dicts("quantity_from_entworpe_to_caleopia_final", \
                                                                   months, lowBound=0, cat='Integer')
quantity_from_entworpe_to_fardo_final_vars = LpVariable.dicts("quantity_from_entworpe_to_fardo_final", \
                                                                   months, lowBound=0, cat='Integer')
quantity_from_entworpe_to_tyran_final_vars = LpVariable.dicts("quantity_from_entworpe_to_tyran_final", \
                                                                   months, lowBound=0, cat='Integer')
quantity_from_entworpe_to_entworpe_final_vars = LpVariable.dicts("quantity_from_entworpe_to_entworpe_final", \
                                                                   months, lowBound=0, cat='Integer')
quantity_from_entworpe_to_sorange_final_vars = LpVariable.dicts("quantity_from_entworpe_to_sorange_final", \
                                                                   months, lowBound=0, cat='Integer')

# quantities sent from tyran warehouse
quantity_from_tyran_to_caleopia_final_vars = LpVariable.dicts("quantity_from_tyran_to_caleopia_final", \
                                                                   months, lowBound=0, cat='Integer')
quantity_from_tyran_to_fardo_final_vars = LpVariable.dicts("quantity_from_tyran_to_fardo_final", \
                                                                   months, lowBound=0, cat='Integer')
quantity_from_tyran_to_tyran_final_vars = LpVariable.dicts("quantity_from_tyran_to_tyran_final", \
                                                                   months, lowBound=0, cat='Integer')
quantity_from_tyran_to_entworpe_final_vars = LpVariable.dicts("quantity_from_tyran_to_entworpe_final", \
                                                                   months, lowBound=0, cat='Integer')
quantity_from_tyran_to_sorange_final_vars = LpVariable.dicts("quantity_from_tyran_to_sorange_final", \
                                                                   months, lowBound=0, cat='Integer')

# initialise the objective function
production_cost = 12*1500 + lpSum([i for i in caleopia_drums_quantity_vars.values()]) * 1000 + \
                  12*1500 + lpSum([i for i in fardo_drums_quantity_vars.values()]) * 1000

transport_from_factory_cost = 15000*lpSum([i for i in truck_quantity_from_caleopia_to_caleopia_vars.values()]) +\
                              45000*lpSum([i for i in truck_quantity_from_fardo_to_caleopia_vars.values()]) + \
                              150*lpSum([i for i in quantity_from_caleopia_to_caleopia_by_mail_vars.values()]) + \
                              400*lpSum([i for i in quantity_from_fardo_to_caleopia_by_mail_vars.values()]) + \
                              20000*lpSum([i for i in truck_quantity_from_caleopia_to_sorange_vars.values()]) +\
                              45000*lpSum([i for i in truck_quantity_from_fardo_to_sorange_vars.values()]) + \
                              200*lpSum([i for i in quantity_from_caleopia_to_sorange_by_mail_vars.values()]) + \
                              400*lpSum([i for i in quantity_from_fardo_to_sorange_by_mail_vars.values()]) + \
                              45000*lpSum([i for i in truck_quantity_from_caleopia_to_fardo_vars.values()]) +\
                              15000*lpSum([i for i in truck_quantity_from_fardo_to_fardo_vars.values()]) + \
                              400*lpSum([i for i in quantity_from_caleopia_to_fardo_by_mail_vars.values()]) + \
                              150*lpSum([i for i in quantity_from_fardo_to_fardo_by_mail_vars.values()]) + \
                              20000*lpSum([i for i in truck_quantity_from_caleopia_to_tyran_vars.values()]) +\
                              45000*lpSum([i for i in truck_quantity_from_fardo_to_tyran_vars.values()]) + \
                              200*lpSum([i for i in quantity_from_caleopia_to_tyran_by_mail_vars.values()]) + \
                              400*lpSum([i for i in quantity_from_fardo_to_tyran_by_mail_vars.values()]) + \
                              20000*lpSum([i for i in truck_quantity_from_caleopia_to_entworpe_vars.values()]) +\
                              45000*lpSum([i for i in truck_quantity_from_fardo_to_entworpe_vars.values()]) + \
                              200*lpSum([i for i in quantity_from_caleopia_to_entworpe_by_mail_vars.values()]) + \
                              400*lpSum([i for i in quantity_from_fardo_to_entworpe_by_mail_vars.values()])

holding_cost = holding_price*(lpSum([i for i in caleopia_end_inventory_vars.values()]) + \
                              lpSum([i for i in quantity_from_caleopia_to_caleopia_by_mail_vars.values()]) + \
                              lpSum([i for i in quantity_from_fardo_to_caleopia_by_mail_vars.values()]) + \
                              lpSum([i for i in quantity_from_caleopia_to_caleopia_by_truck_vars.values()]) + \
                              lpSum([i for i in quantity_from_fardo_to_caleopia_by_truck_vars.values()])) + \
               holding_price*(lpSum([i for i in fardo_end_inventory_vars.values()]) + \
                              lpSum([i for i in quantity_from_caleopia_to_fardo_by_mail_vars.values()]) + \
                              lpSum([i for i in quantity_from_fardo_to_fardo_by_mail_vars.values()]) + \
                              lpSum([i for i in quantity_from_caleopia_to_fardo_by_truck_vars.values()]) + \
                              lpSum([i for i in quantity_from_fardo_to_fardo_by_truck_vars.values()])) + \
               holding_price*(lpSum([i for i in sorange_end_inventory_vars.values()]) + \
                              lpSum([i for i in quantity_from_caleopia_to_sorange_by_mail_vars.values()]) + \
                              lpSum([i for i in quantity_from_fardo_to_sorange_by_mail_vars.values()]) + \
                              lpSum([i for i in quantity_from_caleopia_to_sorange_by_truck_vars.values()]) + \
                              lpSum([i for i in quantity_from_fardo_to_sorange_by_truck_vars.values()])) + \
               holding_price*(lpSum([i for i in tyran_end_inventory_vars.values()]) + \
                              lpSum([i for i in quantity_from_caleopia_to_tyran_by_mail_vars.values()]) + \
                              lpSum([i for i in quantity_from_fardo_to_tyran_by_mail_vars.values()]) + \
                              lpSum([i for i in quantity_from_caleopia_to_tyran_by_truck_vars.values()]) + \
                              lpSum([i for i in quantity_from_fardo_to_tyran_by_truck_vars.values()])) + \
               holding_price*(lpSum([i for i in entworpe_end_inventory_vars.values()]) + \
                              lpSum([i for i in quantity_from_caleopia_to_entworpe_by_mail_vars.values()]) + \
                              lpSum([i for i in quantity_from_fardo_to_entworpe_by_mail_vars.values()]) + \
                              lpSum([i for i in quantity_from_caleopia_to_entworpe_by_truck_vars.values()]) + \
                              lpSum([i for i in quantity_from_fardo_to_entworpe_by_truck_vars.values()]))

transport_from_warehouse_cost = 150*lpSum([i for i in quantity_from_fardo_to_fardo_final_vars.values()]) + \
                                150*lpSum([i for i in quantity_from_caleopia_to_caleopia_final_vars.values()]) + \
                                200*lpSum([i for i in quantity_from_caleopia_to_sorange_final_vars.values()]) + \
                                200*lpSum([i for i in quantity_from_caleopia_to_tyran_final_vars.values()]) + \
                                200*lpSum([i for i in quantity_from_caleopia_to_entworpe_final_vars.values()]) + \
                                200*lpSum([i for i in quantity_from_tyran_to_caleopia_final_vars.values()]) + \
                                200*lpSum([i for i in quantity_from_tyran_to_sorange_final_vars.values()]) + \
                                200*lpSum([i for i in quantity_from_tyran_to_entworpe_final_vars.values()]) + \
                                150*lpSum([i for i in quantity_from_tyran_to_tyran_final_vars.values()]) + \
                                200*lpSum([i for i in quantity_from_sorange_to_caleopia_final_vars.values()]) + \
                                150*lpSum([i for i in quantity_from_sorange_to_sorange_final_vars.values()]) + \
                                200*lpSum([i for i in quantity_from_sorange_to_tyran_final_vars.values()]) + \
                                200*lpSum([i for i in quantity_from_sorange_to_entworpe_final_vars.values()]) + \
                                200*lpSum([i for i in quantity_from_entworpe_to_caleopia_final_vars.values()]) + \
                                200*lpSum([i for i in quantity_from_entworpe_to_sorange_final_vars.values()]) + \
                                200*lpSum([i for i in quantity_from_entworpe_to_tyran_final_vars.values()]) + \
                                150*lpSum([i for i in quantity_from_entworpe_to_entworpe_final_vars.values()])
                                

total_cost = LpProblem("Supply_Chain_Game_Final", LpMinimize)
total_cost += production_cost + transport_from_factory_cost + holding_cost + transport_from_warehouse_cost

# constraints
for j in range(0,3):
    total_cost += [i for i in caleopia_drums_quantity_vars.values()][j] <= 70*30
    total_cost += [i for i in fardo_drums_quantity_vars.values()][j] == 0
    
for j in range(3,24):
    total_cost += [i for i in caleopia_drums_quantity_vars.values()][j] <= 145*30
    total_cost += [i for i in fardo_drums_quantity_vars.values()][j] <= 17*30
    
for j in range(0,24):
    total_cost += [i for i in quantity_from_caleopia_to_caleopia_by_truck_vars.values()][j] \
                    <= 200*[i for i in truck_quantity_from_caleopia_to_caleopia_vars.values()][j]
    total_cost += [i for i in quantity_from_fardo_to_caleopia_by_truck_vars.values()][j] \
                    <= 200*[i for i in truck_quantity_from_fardo_to_caleopia_vars.values()][j]
    total_cost += [i for i in quantity_from_caleopia_to_caleopia_by_mail_vars.values()][j] \
                    <= 10000
    total_cost += [i for i in quantity_from_fardo_to_caleopia_by_mail_vars.values()][j] \
                    <= 10000
        
    total_cost += [i for i in quantity_from_caleopia_to_sorange_by_truck_vars.values()][j] \
                    <= 200*[i for i in truck_quantity_from_caleopia_to_sorange_vars.values()][j]
    total_cost += [i for i in quantity_from_fardo_to_sorange_by_truck_vars.values()][j] \
                    <= 200*[i for i in truck_quantity_from_fardo_to_sorange_vars.values()][j]
    total_cost += [i for i in quantity_from_caleopia_to_sorange_by_mail_vars.values()][j] \
                    <= 10000
    total_cost += [i for i in quantity_from_fardo_to_sorange_by_mail_vars.values()][j] \
                    <= 10000
        
    total_cost += [i for i in quantity_from_caleopia_to_entworpe_by_truck_vars.values()][j] \
                    <= 200*[i for i in truck_quantity_from_caleopia_to_entworpe_vars.values()][j]
    total_cost += [i for i in quantity_from_fardo_to_entworpe_by_truck_vars.values()][j] \
                    <= 200*[i for i in truck_quantity_from_fardo_to_entworpe_vars.values()][j]
    total_cost += [i for i in quantity_from_caleopia_to_entworpe_by_mail_vars.values()][j] \
                    <= 10000
    total_cost += [i for i in quantity_from_fardo_to_entworpe_by_mail_vars.values()][j] \
                    <= 10000
        
    total_cost += [i for i in quantity_from_caleopia_to_fardo_by_truck_vars.values()][j] \
                    <= 200*[i for i in truck_quantity_from_caleopia_to_fardo_vars.values()][j]
    total_cost += [i for i in quantity_from_fardo_to_fardo_by_truck_vars.values()][j] \
                    <= 200*[i for i in truck_quantity_from_fardo_to_fardo_vars.values()][j]
    total_cost += [i for i in quantity_from_caleopia_to_fardo_by_mail_vars.values()][j] \
                    <= 10000
    total_cost += [i for i in quantity_from_fardo_to_fardo_by_mail_vars.values()][j] \
                    <= 10000
    
    total_cost += [i for i in quantity_from_caleopia_to_tyran_by_truck_vars.values()][j] \
                    <= 200*[i for i in truck_quantity_from_caleopia_to_tyran_vars.values()][j]
    total_cost += [i for i in quantity_from_fardo_to_tyran_by_truck_vars.values()][j] \
                    <= 200*[i for i in truck_quantity_from_fardo_to_tyran_vars.values()][j]
    total_cost += [i for i in quantity_from_caleopia_to_tyran_by_mail_vars.values()][j] \
                    <= 10000
    total_cost += [i for i in quantity_from_fardo_to_tyran_by_mail_vars.values()][j] \
                    <= 10000
    
for j in range(0, 2):
    total_cost += [i for i in quantity_from_caleopia_to_sorange_by_mail_vars.values()][j] == 0
    total_cost += [i for i in truck_quantity_from_caleopia_to_sorange_vars.values()][j] == 0
    total_cost += [i for i in quantity_from_fardo_to_sorange_by_mail_vars.values()][j] == 0
    total_cost += [i for i in truck_quantity_from_fardo_to_sorange_vars.values()][j] == 0

    total_cost += [i for i in quantity_from_caleopia_to_fardo_by_mail_vars.values()][j] == 0
    total_cost += [i for i in truck_quantity_from_caleopia_to_fardo_vars.values()][j] == 0
    total_cost += [i for i in quantity_from_fardo_to_fardo_by_mail_vars.values()][j] == 0
    total_cost += [i for i in truck_quantity_from_fardo_to_fardo_vars.values()][j] == 0

    total_cost += [i for i in quantity_from_caleopia_to_tyran_by_mail_vars.values()][j] == 0
    total_cost += [i for i in truck_quantity_from_caleopia_to_tyran_vars.values()][j] == 0
    total_cost += [i for i in quantity_from_fardo_to_tyran_by_mail_vars.values()][j] == 0
    total_cost += [i for i in truck_quantity_from_fardo_to_tyran_vars.values()][j] == 0

    total_cost += [i for i in quantity_from_caleopia_to_entworpe_by_mail_vars.values()][j] == 0
    total_cost += [i for i in truck_quantity_from_caleopia_to_entworpe_vars.values()][j] == 0
    total_cost += [i for i in quantity_from_fardo_to_entworpe_by_mail_vars.values()][j] == 0
    total_cost += [i for i in truck_quantity_from_fardo_to_entworpe_vars.values()][j] == 0
    
for j in range(0,24):
    total_cost += ([i for i in caleopia_drums_quantity_vars.values()][j] == \
                  [i for i in quantity_from_caleopia_to_entworpe_by_mail_vars.values()][j] + \
                  [i for i in quantity_from_caleopia_to_entworpe_by_truck_vars.values()][j] + \
                  [i for i in quantity_from_caleopia_to_sorange_by_mail_vars.values()][j] + \
                  [i for i in quantity_from_caleopia_to_sorange_by_truck_vars.values()][j] + \
                  [i for i in quantity_from_caleopia_to_fardo_by_mail_vars.values()][j] + \
                  [i for i in quantity_from_caleopia_to_fardo_by_truck_vars.values()][j] + \
                  [i for i in quantity_from_caleopia_to_tyran_by_mail_vars.values()][j] + \
                  [i for i in quantity_from_caleopia_to_tyran_by_truck_vars.values()][j] + \
                  [i for i in quantity_from_caleopia_to_caleopia_by_mail_vars.values()][j] + \
                  [i for i in quantity_from_caleopia_to_caleopia_by_truck_vars.values()][j]) 
    
    total_cost += ([i for i in fardo_drums_quantity_vars.values()][j] == \
                  [i for i in quantity_from_fardo_to_entworpe_by_mail_vars.values()][j] + \
                  [i for i in quantity_from_fardo_to_entworpe_by_truck_vars.values()][j] + \
                  [i for i in quantity_from_fardo_to_sorange_by_mail_vars.values()][j] + \
                  [i for i in quantity_from_fardo_to_sorange_by_truck_vars.values()][j] + \
                  [i for i in quantity_from_fardo_to_fardo_by_mail_vars.values()][j] + \
                  [i for i in quantity_from_fardo_to_fardo_by_truck_vars.values()][j] + \
                  [i for i in quantity_from_fardo_to_tyran_by_mail_vars.values()][j] + \
                  [i for i in quantity_from_fardo_to_tyran_by_truck_vars.values()][j] + \
                  [i for i in quantity_from_fardo_to_caleopia_by_mail_vars.values()][j] + \
                  [i for i in quantity_from_fardo_to_caleopia_by_truck_vars.values()][j]) 

for j in range(0,2):
    total_cost += [i for i in entworpe_end_inventory_vars.values()][j] == 0
    total_cost += [i for i in sorange_end_inventory_vars.values()][j] == 0
    total_cost += [i for i in fardo_end_inventory_vars.values()][j] == 0
    total_cost += [i for i in tyran_end_inventory_vars.values()][j] == 0

total_cost += [i for i in caleopia_start_inventory_vars.values()][0] == 2528

for j in range(0,23):
    total_cost += [i for i in entworpe_end_inventory_vars.values()][j] == \
                  [i for i in entworpe_start_inventory_vars.values()][j+1]
    total_cost += [i for i in caleopia_end_inventory_vars.values()][j] == \
                  [i for i in caleopia_start_inventory_vars.values()][j+1]
    total_cost += [i for i in fardo_end_inventory_vars.values()][j] == \
                  [i for i in fardo_start_inventory_vars.values()][j+1]
    total_cost += [i for i in tyran_end_inventory_vars.values()][j] == \
                  [i for i in tyran_start_inventory_vars.values()][j+1]
    total_cost += [i for i in sorange_end_inventory_vars.values()][j] == \
                  [i for i in sorange_start_inventory_vars.values()][j+1]

for j in range(0,2):
    total_cost += [i for i in quantity_from_fardo_to_fardo_final_vars.values()][j] == 0
    
    total_cost += [i for i in quantity_from_sorange_to_sorange_final_vars.values()][j] == 0
    total_cost += [i for i in quantity_from_sorange_to_caleopia_final_vars.values()][j] == 0
    total_cost += [i for i in quantity_from_sorange_to_tyran_final_vars.values()][j] == 0
    total_cost += [i for i in quantity_from_sorange_to_entworpe_final_vars.values()][j] == 0
    
    total_cost += [i for i in quantity_from_tyran_to_sorange_final_vars.values()][j] == 0
    total_cost += [i for i in quantity_from_tyran_to_caleopia_final_vars.values()][j] == 0
    total_cost += [i for i in quantity_from_tyran_to_tyran_final_vars.values()][j] == 0
    total_cost += [i for i in quantity_from_tyran_to_entworpe_final_vars.values()][j] == 0
    
    total_cost += [i for i in quantity_from_entworpe_to_sorange_final_vars.values()][j] == 0
    total_cost += [i for i in quantity_from_entworpe_to_caleopia_final_vars.values()][j] == 0
    total_cost += [i for i in quantity_from_entworpe_to_tyran_final_vars.values()][j] == 0
    total_cost += [i for i in quantity_from_entworpe_to_entworpe_final_vars.values()][j] == 0

for j in range(3,24):
    total_cost += [i for i in quantity_from_fardo_to_fardo_final_vars.values()][j] >= demand_fardo[j]

for j in range(0,24):
    total_cost += ([i for i in quantity_from_entworpe_to_sorange_final_vars.values()][j] + \
                        [i for i in quantity_from_caleopia_to_sorange_final_vars.values()][j] + \
                        [i for i in quantity_from_tyran_to_sorange_final_vars.values()][j] + \
                        [i for i in quantity_from_sorange_to_sorange_final_vars.values()][j]) >= demand_sorange[j]
    
    total_cost += ([i for i in quantity_from_entworpe_to_entworpe_final_vars.values()][j] + \
                        [i for i in quantity_from_caleopia_to_entworpe_final_vars.values()][j] + \
                        [i for i in quantity_from_tyran_to_entworpe_final_vars.values()][j] + \
              [i for i in quantity_from_sorange_to_entworpe_final_vars.values()][j]) >= demand_entworpe2.values[j]
    
    total_cost += ([i for i in quantity_from_entworpe_to_caleopia_final_vars.values()][j] + \
                        [i for i in quantity_from_caleopia_to_caleopia_final_vars.values()][j] + \
                        [i for i in quantity_from_tyran_to_caleopia_final_vars.values()][j] + \
                        [i for i in quantity_from_sorange_to_caleopia_final_vars.values()][j]) >= demand_calopeia[j]
    
    total_cost += ([i for i in quantity_from_entworpe_to_tyran_final_vars.values()][j] + \
                        [i for i in quantity_from_caleopia_to_tyran_final_vars.values()][j] + \
                        [i for i in quantity_from_tyran_to_tyran_final_vars.values()][j] + \
                        [i for i in quantity_from_sorange_to_tyran_final_vars.values()][j]) >= demand_tyran[j]
    
for j in range(0,24):
    total_cost += [i for i in caleopia_end_inventory_vars.values()][j] == \
                  [i for i in caleopia_start_inventory_vars.values()][j] + \
                  [i for i in quantity_from_caleopia_to_caleopia_by_truck_vars.values()][j] + \
                  [i for i in quantity_from_fardo_to_caleopia_by_truck_vars.values()][j] + \
                  [i for i in quantity_from_caleopia_to_caleopia_by_mail_vars.values()][j] + \
                  [i for i in quantity_from_fardo_to_caleopia_by_mail_vars.values()][j] - \
                  ([i for i in quantity_from_caleopia_to_caleopia_final_vars.values()][j] + \
                  [i for i in quantity_from_caleopia_to_sorange_final_vars.values()][j] + \
                  [i for i in quantity_from_caleopia_to_entworpe_final_vars.values()][j] + \
                  [i for i in quantity_from_caleopia_to_tyran_final_vars.values()][j])

    total_cost += [i for i in sorange_end_inventory_vars.values()][j] == \
                  [i for i in sorange_start_inventory_vars.values()][j] + \
                  [i for i in quantity_from_caleopia_to_sorange_by_truck_vars.values()][j] + \
                  [i for i in quantity_from_fardo_to_sorange_by_truck_vars.values()][j] + \
                  [i for i in quantity_from_caleopia_to_sorange_by_mail_vars.values()][j] + \
                  [i for i in quantity_from_fardo_to_sorange_by_mail_vars.values()][j] - \
                  ([i for i in quantity_from_sorange_to_caleopia_final_vars.values()][j] + \
                  [i for i in quantity_from_sorange_to_sorange_final_vars.values()][j] + \
                  [i for i in quantity_from_sorange_to_entworpe_final_vars.values()][j] + \
                  [i for i in quantity_from_sorange_to_tyran_final_vars.values()][j])
    
    total_cost += [i for i in entworpe_end_inventory_vars.values()][j] == \
                  [i for i in entworpe_start_inventory_vars.values()][j] + \
                  [i for i in quantity_from_caleopia_to_entworpe_by_truck_vars.values()][j] + \
                  [i for i in quantity_from_fardo_to_entworpe_by_truck_vars.values()][j] + \
                  [i for i in quantity_from_caleopia_to_entworpe_by_mail_vars.values()][j] + \
                  [i for i in quantity_from_fardo_to_entworpe_by_mail_vars.values()][j] - \
                  ([i for i in quantity_from_entworpe_to_caleopia_final_vars.values()][j] + \
                  [i for i in quantity_from_entworpe_to_sorange_final_vars.values()][j] + \
                  [i for i in quantity_from_entworpe_to_entworpe_final_vars.values()][j] + \
                  [i for i in quantity_from_entworpe_to_tyran_final_vars.values()][j])
    
    total_cost += [i for i in tyran_end_inventory_vars.values()][j] == \
                  [i for i in tyran_start_inventory_vars.values()][j] + \
                  [i for i in quantity_from_caleopia_to_tyran_by_truck_vars.values()][j] + \
                  [i for i in quantity_from_fardo_to_tyran_by_truck_vars.values()][j] + \
                  [i for i in quantity_from_caleopia_to_tyran_by_mail_vars.values()][j] + \
                  [i for i in quantity_from_fardo_to_tyran_by_mail_vars.values()][j] - \
                  ([i for i in quantity_from_tyran_to_caleopia_final_vars.values()][j] + \
                  [i for i in quantity_from_tyran_to_sorange_final_vars.values()][j] + \
                  [i for i in quantity_from_tyran_to_entworpe_final_vars.values()][j] + \
                  [i for i in quantity_from_tyran_to_tyran_final_vars.values()][j])
    
    total_cost += [i for i in fardo_end_inventory_vars.values()][j] == \
                  [i for i in fardo_start_inventory_vars.values()][j] + \
                  [i for i in quantity_from_caleopia_to_fardo_by_truck_vars.values()][j] + \
                  [i for i in quantity_from_fardo_to_fardo_by_truck_vars.values()][j] + \
                  [i for i in quantity_from_caleopia_to_fardo_by_mail_vars.values()][j] + \
                  [i for i in quantity_from_fardo_to_fardo_by_mail_vars.values()][j] - \
                  ([i for i in quantity_from_fardo_to_fardo_final_vars.values()][j])

solver = CPLEX_CMD(path='/Applications/CPLEX_Studio221/cplex/bin/x86-64_osx/cplex')
total_cost.solve(solver)


Welcome to IBM(R) ILOG(R) CPLEX(R) Interactive Optimizer 22.1.0.0
  with Simplex, Mixed Integer & Barrier Optimizers
5725-A06 5725-A29 5724-Y48 5724-Y49 5724-Y54 5724-Y55 5655-Y21
Copyright IBM Corp. 1988, 2022.  All Rights Reserved.

Type 'help' for a list of available commands.
Type 'help' followed by a command name for more
information on commands.

CPLEX> Problem '/var/folders/24/642ljpss6bd8ljpp58f9qjy40000gn/T/ee66763449d4444db55e9ade819d984b-pulp.lp' read.
Read time = 0.02 sec. (0.11 ticks)
CPLEX> Version identifier: 22.1.0.0 | 2022-03-09 | 1a383f8ce
Tried aggregator 2 times.
MIP Presolve eliminated 426 rows and 126 columns.
MIP Presolve modified 44 coefficients.
Aggregator did 107 substitutions.
Reduced MIP has 462 rows, 1183 columns, and 2293 nonzeros.
Reduced MIP has 0 binaries, 1183 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.01 sec. (3.35 ticks)
Found incumbent of value 2.0491428e+08 after 0.05 sec. (11.85 ticks)
Tried aggregator 1 time.
Detecting symmetries...
R

 560931 323892   1.06009e+08    77   1.06019e+08   1.06006e+08  3846623    0.01%
 582015 335653   1.06019e+08    94   1.06019e+08   1.06006e+08  4033608    0.01%
 604466 348110   1.06019e+08    44   1.06019e+08   1.06006e+08  4235427    0.01%
 625837 361526   1.06017e+08   115   1.06019e+08   1.06007e+08  4414219    0.01%
 642727 369138   1.06019e+08    41   1.06019e+08   1.06007e+08  4534730    0.01%
 663996 378506   1.06018e+08   107   1.06019e+08   1.06007e+08  4709584    0.01%
 683144 388516   1.06011e+08   104   1.06019e+08   1.06007e+08  4890528    0.01%
 709715 398339   1.06019e+08   118   1.06019e+08   1.06007e+08  5042702    0.01%
 735753 410974        cutoff         1.06019e+08   1.06007e+08  5279033    0.01%
 760957 420883        cutoff         1.06019e+08   1.06007e+08  5468241    0.01%
Elapsed time = 81.85 sec. (32187.72 ticks, tree = 1552.09 MB, solutions = 44)
 786350 433502   1.06018e+08   103   1.06019e+08   1.06008e+08  5713831    0.01%
 811331 441947   1.06013e+08   

1

In [500]:
print("Status:", LpStatus[total_cost.status])
for v in total_cost.variables():
    if v.varValue > 0:
        print(v.name, "=", v.varValue)

Status: Optimal
caleopia_drums_quantity_10 = 3000.0000000000305
caleopia_drums_quantity_11 = 3200.0
caleopia_drums_quantity_12 = 4200.0
caleopia_drums_quantity_13 = 4200.0
caleopia_drums_quantity_14 = 4199.999999999891
caleopia_drums_quantity_15 = 4223.0
caleopia_drums_quantity_16 = 4350.0
caleopia_drums_quantity_17 = 4350.0
caleopia_drums_quantity_18 = 4350.0
caleopia_drums_quantity_19 = 4350.0
caleopia_drums_quantity_2 = 600.0
caleopia_drums_quantity_20 = 4350.0
caleopia_drums_quantity_21 = 4350.0
caleopia_drums_quantity_22 = 4350.0
caleopia_drums_quantity_23 = 4350.0
caleopia_drums_quantity_24 = 2589.0
caleopia_drums_quantity_3 = 2000.0000000000273
caleopia_drums_quantity_4 = 2599.999999999993
caleopia_drums_quantity_5 = 3000.0
caleopia_drums_quantity_6 = 4000.0
caleopia_drums_quantity_7 = 3800.0000000000578
caleopia_drums_quantity_8 = 3600.0
caleopia_drums_quantity_9 = 3800.0
caleopia_end_inventory_1 = 1204.0
caleopia_end_inventory_10 = 147.0
caleopia_end_inventory_11 = 116.0
caleo

### Caleopia - Monthly ROP 

In [502]:
calopeia_demand3 = pd.DataFrame(index =[i for i in range(1,13)]*2)
L = 7

D_cc = []
for i in range(0,24):
    for v in total_cost.variables():
        if v.name == f"quantity_from_caleopia_to_caleopia_final_{i+1}":
            D_cc.append(v.varValue)
            
D_ct = []
for i in range(0,24):
    for v in total_cost.variables():
        if v.name == f"quantity_from_caleopia_to_tyran_final_{i+1}":
            D_ct.append(v.varValue)

D_cs = []
for i in range(0,24):
    for v in total_cost.variables():
        if v.name == f"quantity_from_caleopia_to_sorange_final_{i+1}":
            D_cs.append(v.varValue)
            
D_ce = []
for i in range(0,24):
    for v in total_cost.variables():
        if v.name == f"quantity_from_caleopia_to_entworpe_final_{i+1}":
            D_ce.append(v.varValue)
        
calopeia_demand3['D_cc'] = D_cc
calopeia_demand3['D_ct'] = D_ct
calopeia_demand3['D_cs'] = D_cs
calopeia_demand3['D_ce'] = D_ce
calopeia_demand3['avg_daily_D'] = calopeia_demand3.iloc[:, :].sum(axis=1)/30
calopeia_demand3['POQ_caleopia'] = calopeia_demand3.iloc[:, :].sum(axis=1)
calopeia_demand3['ROP_caleopia'] = L_caleopia*calopeia_demand3['avg_daily_D'] + 250
calopeia_demand_final = calopeia_demand3.loc[:, ['POQ_caleopia','ROP_caleopia']]
calopeia_demand_final

,POQ_caleopia,ROP_caleopia
1,1368.133333,558.933333
2,1349.533333,554.733333
3,1061.233333,489.633333
4,1401.200000,566.400000
5,1978.833333,696.833333
6,2214.433333,750.033333
7,2309.500000,771.500000
8,2068.733333,717.133333
9,1617.166667,615.166667
10,938.266667,461.866667


### Sorange - Monthly EOQ and ROP 

In [462]:
sorange_demand2 = pd.DataFrame(index =[i for i in range(1,13)]*2)

D_sc = []
for i in range(0,24):
    for v in total_cost.variables():
        if v.name == f"quantity_from_sorange_to_caleopia_final_{i+1}":
            D_sc.append(v.varValue)
            
D_st = []
for i in range(0,24):
    for v in total_cost.variables():
        if v.name == f"quantity_from_sorange_to_tyran_final_{i+1}":
            D_st.append(v.varValue)

D_ss = []
for i in range(0,24):
    for v in total_cost.variables():
        if v.name == f"quantity_from_sorange_to_sorange_final_{i+1}":
            D_ss.append(v.varValue)
            
D_se = []
for i in range(0,24):
    for v in total_cost.variables():
        if v.name == f"quantity_from_sorange_to_entworpe_final_{i+1}":
            D_se.append(v.varValue)
        
sorange_demand2['D_sc'] = D_sc
sorange_demand2['D_st'] = D_st
sorange_demand2['D_ss'] = D_ss
sorange_demand2['D_se'] = D_se
sorange_demand2['avg_daily_D'] = sorange_demand2.iloc[:, :].sum(axis=1)/30
sorange_demand2['POQ_sorange'] = sorange_demand2.iloc[:, :].sum(axis=1)/30
sorange_demand2['ROP_sorange'] = L*sorange_demand2['avg_daily_D'] + 250
sorange_demand_final = sorange_demand2.loc[:, ['POQ_sorange','ROP_sorange']]

### Tyran - Monthly ROP 

In [463]:
tyran_demand = pd.DataFrame(index =[i for i in range(1,13)]*2)

D_tc = []
for i in range(0,24):
    for v in total_cost.variables():
        if v.name == f"quantity_from_tyran_to_caleopia_final_{i+1}":
            D_tc.append(v.varValue)
            
D_tt = []
for i in range(0,24):
    for v in total_cost.variables():
        if v.name == f"quantity_from_tyran_to_tyran_final_{i+1}":
            D_tt.append(v.varValue)

D_ts = []
for i in range(0,24):
    for v in total_cost.variables():
        if v.name == f"quantity_from_tyran_to_sorange_final_{i+1}":
            D_ts.append(v.varValue)
            
D_te = []
for i in range(0,24):
    for v in total_cost.variables():
        if v.name == f"quantity_from_tyran_to_entworpe_final_{i+1}":
            D_te.append(v.varValue)
        
tyran_demand['D_tc'] = D_tc
tyran_demand['D_tt'] = D_tt
tyran_demand['D_ts'] = D_ts
tyran_demand['D_te'] = D_te
tyran_demand['avg_daily_D'] = tyran_demand.iloc[:, :].sum(axis=1)/30
tyran_demand['POQ_tyran'] = tyran_demand.iloc[:, :].sum(axis=1)
tyran_demand['ROP_tyran'] = L*tyran_demand['avg_daily_D'] + 0.1*L*tyran_demand['avg_daily_D']
tyran_demand_final = tyran_demand.loc[:, ['POQ_tyran','ROP_tyran']]

### Entworpe - Monthly ROP 

In [464]:
entworpe_demand = pd.DataFrame(index =[i for i in range(1,13)]*2)

D_ec = []
for i in range(0,24):
    for v in total_cost.variables():
        if v.name == f"quantity_from_entworpe_to_caleopia_final_{i+1}":
            D_ec.append(v.varValue)
            
D_et = []
for i in range(0,24):
    for v in total_cost.variables():
        if v.name == f"quantity_from_entworpe_to_tyran_final_{i+1}":
            D_et.append(v.varValue)

D_es = []
for i in range(0,24):
    for v in total_cost.variables():
        if v.name == f"quantity_from_entworpe_to_sorange_final_{i+1}":
            D_es.append(v.varValue)
            
D_ee = []
for i in range(0,24):
    for v in total_cost.variables():
        if v.name == f"quantity_from_entworpe_to_entworpe_final_{i+1}":
            D_ee.append(v.varValue)
        
entworpe_demand['D_ec'] = D_ec
entworpe_demand['D_et'] = D_et
entworpe_demand['D_es'] = D_es
entworpe_demand['D_ee'] = D_ee
entworpe_demand['avg_daily_D'] = entworpe_demand.iloc[:, :].sum(axis=1)/30
entworpe_demand['POQ_entworpe'] = entworpe_demand.iloc[:, :].sum(axis=1)
entworpe_demand['ROP_entworpe'] = L*entworpe_demand['avg_daily_D'] + 0.1*L*entworpe_demand['avg_daily_D']
entworpe_demand_final = entworpe_demand.loc[:, ['POQ_entworpe','ROP_entworpe']]

### Fardo - Monthly ROP 

In [465]:
fardo_demand = pd.DataFrame(index =[i for i in range(1,13)]*2)

D_ff = []
for i in range(0,24):
    for v in total_cost.variables():
        if v.name == f"quantity_from_fardo_to_fardo_final_{i+1}":
            D_ff.append(v.varValue)
        
fardo_demand['D_ff'] = D_ff
fardo_demand['avg_daily_D'] = fardo_demand.iloc[:, :].sum(axis=1)/30
fardo_demand['POQ_fardo'] = fardo_demand.iloc[:, :].sum(axis=1)
fardo_demand['ROP_fardo'] = L*fardo_demand['avg_daily_D'] + 0.1*L*fardo_demand['avg_daily_D']
fardo_demand_final = fardo_demand.loc[:, ['POQ_fardo','ROP_fardo']]

In [466]:
Game_parameters = pd.concat([calopeia_demand_final, sorange_demand_final, tyran_demand_final, entworpe_demand_final, fardo_demand_final], axis=1)
Game_parameters

,POQ_caleopia,ROP_caleopia,POQ_sorange,ROP_sorange,POQ_tyran,ROP_tyran,POQ_entworpe,ROP_entworpe,POQ_fardo,ROP_fardo
1,862.833333,444.833333,0.000000,250.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000
2,1102.566667,498.966667,0.000000,250.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000
3,1060.200000,489.400000,27.417778,435.733333,0.000000,0.000000,0.0,0.0,0.000000,0.000000
4,1433.233333,573.633333,30.965556,459.766667,0.000000,0.000000,0.0,0.0,16.533333,4.106667
5,2027.400000,707.800000,36.407778,496.633333,0.000000,0.000000,0.0,0.0,16.533333,4.106667
6,2225.800000,752.600000,40.575556,524.866667,0.000000,0.000000,0.0,0.0,16.533333,4.106667
7,2320.866667,774.066667,46.913333,567.800000,0.000000,0.000000,0.0,0.0,16.533333,4.106667
8,2034.633333,709.433333,51.942222,601.866667,0.000000,0.000000,0.0,0.0,16.533333,4.106667
9,1628.533333,617.733333,55.076667,623.100000,0.000000,0.000000,0.0,0.0,16.533333,4.106667
10,936.200000,461.400000,62.034444,670.233333,9.300000,2.310000,0.0,0.0,16.533333,4.106667


In [456]:
Game_parameters.to_excel('game_parameters.xlsx')